## NACA 0012 Airfoil

In [1]:
import numpy
import math
from matplotlib import pyplot
%matplotlib inline

In [2]:
N = 51
x_start, x_end = -1.0,2.0
y_start, y_end = -0.5,0.5
x = numpy.linspace(x_start, x_end, N)
y = numpy.linspace(y_start, y_end, N)
X, Y = numpy.meshgrid(x, y) 

In [3]:
#set freestream speed
u_inf = 1.0

v_freestream = numpy.zeros((N, N), dtype=float)
u_freestream = u_inf * numpy.ones((N, N), dtype=float)